In [1]:
from decomp_utils import *
import tensorflow as tf
import numpy as np
import tensorflow_addons as tfa
from sklearn.metrics import confusion_matrix
import tensorflow_probability as tfp

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dropout, Conv2D, MaxPooling2D, BatchNormalization

D:\anaconda\envs\decomp\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.7.0 and strictly below 2.10.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [2]:
path = "D:\\AI-Workspace\\Decomposition\\ViT_Decompose\\data\\5_50_GM-"
train_ls = [1, 3]
test_ls = 2
overlap = 5
x_sEMG = 'EMGs'
y_spikes = 'Spikes'
mu = [0,1,2,3,4,5,6,7,8,9]

In [3]:
x, y = load_train(path, train_ls, overlap,mu)
x = np.expand_dims(x, axis=3)  

In [4]:
input_shape = x.shape[1:]
output_shape = 1

In [5]:
#Hyper-parameter

input_shape = input_shape
patch_size = 4
no_patches = 240 #(x.shape[1] // patch_size) * (x.shape[2] // patch_size)
dims = 100
no_transformer_layers = 4
no_heads = 2
transformer_units = [dims * 2,dims]  # Size of the transformer layers
mlp_head_units = [240, 10]   # [2048, 1024]
num_classes = 1
learning_rate = 0.001
weight_decay = 0.0001

In [6]:
class Generate_Patches(tf.keras.layers.Layer):
    
    def __init__(self, patch_size):
        super(Generate_Patches, self).__init__()
        self.patch_size = patch_size
    
    
    def call(self, images):
        
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(images=images,
                           sizes=[1, self.patch_size, self.patch_size, 1],
                           strides=[1, self.patch_size, self.patch_size, 1],
                           rates=[1, 1, 1, 1],
                           padding='VALID')
        h_w_c = patches.shape[-1]
        patch_reshape = tf.reshape(patches, [batch_size, -1, h_w_c]) # (batch_size, no_of_patches, h*w*c)
        
        return patch_reshape

In [7]:
class Embed_Position(tf.keras.layers.Layer):
    
    def __init__(self, num_patches, projection_dims):
        
        super(Embed_Position, self).__init__()
        
        self.num_patches = num_patches
        self.project = tf.keras.layers.Dense(units=projection_dims)
        self.embed = tf.keras.layers.Embedding(input_dim=num_patches, output_dim=projection_dims)
    
    def call(self, patch):
        
        position = tf.range(start=0, limit=self.num_patches, delta=1)
        encode = self.project(patch) + self.embed(position)
        
        return encode

In [8]:
def mlp_mixer(inputs, no_units, drop_out):
    
    for units in no_units:
        x = tf.keras.layers.Dense(units=units, activation = tf.nn.gelu)(inputs)
        x = tf.keras.layers.Dropout(drop_out)(x)
        # x = tf.keras.layers.Dense(inputs.shape[-1], activation=tf.nn.gelu)(x)
    
    return x

In [9]:
conv_layer = keras.Sequential(
    [
        Conv2D(64, (8, 8), activation="relu", padding="same", input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D((1, 2)),
        Dropout(0.3),
        Conv2D(32, (4, 4), activation="relu", padding="same"),
        BatchNormalization(),
        Dropout(0.3),
        Conv2D(dims, (2, 2), activation="relu", padding="same"),
        BatchNormalization(),
    ],
    name="covolution_layer",
)

In [10]:
def mo_vit_trans_decomp():
    
    outputs = []
    no_of_nodes = len(mu)
    
    inputs = layers.Input(shape=input_shape)
    # Token embedding.
    con_emb = conv_layer(inputs)
    # Create patches.
    #patches = Patches(patch_size)(tokenemb)
    # Encode patches.
    #encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)
    
    
    #inputs = tf.keras.Input(shape=input_shape)
    #x = tf.keras.layers.BatchNormalization()(inputs)
    returned_patches = Generate_Patches(patch_size)(con_emb)
    encoded_patches = Embed_Position(no_patches, dims)(returned_patches)
    #encoded_patches = tf.keras.layers.BatchNormalization()(encoded_patches)
    # Create multiple layers of the Transformer block.
    for _ in range(no_transformer_layers):
        # Layer normalization 1.
        input_two_attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = tf.keras.layers.MultiHeadAttention(num_heads=no_heads, key_dim=dims, dropout=0.1)(input_two_attention, input_two_attention)
        # Skip connection 1.
        input_two_attention_2 = tf.keras.layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        input_two_attention_3  = tf.keras.layers.LayerNormalization(epsilon=1e-6)(input_two_attention_2)
        # MLP.
        input_two_attention_3 = mlp_mixer(input_two_attention_3, no_units=transformer_units, drop_out=0.1)
        # Skip connection 2.
        encoded_patches = tf.keras.layers.Add()([input_two_attention_3, input_two_attention_3])
        
        
# Create a [batch_size, projection_dim] tensor.
    representation = tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = tf.keras.layers.Flatten()(representation)
    representation = tf.keras.layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp_mixer(representation, no_units=mlp_head_units, drop_out=0.5)
    # Classify outputs.
    
    for i in range(1, no_of_nodes+1):
        output = tf.keras.layers.Dense(output_shape, activation='sigmoid', name='output_{}'.format(i))(features)
        outputs.append(output)

    # Create the Keras model.
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="sEMG-Decomposition")
    
    return model

In [11]:
model = mo_vit_trans_decomp()
model.summary()

Model: "sEMG-Decomposition"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 120, 64, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 covolution_layer (Sequential)  (None, 120, 32, 100  50644       ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 generate__patches (Generate_Pa  (None, None, 1600)  0           ['covolution_layer[0][0]']       
 tches)                                                                          

                                                                                                  
 dense_8 (Dense)                (None, 240, 100)     10100       ['layer_normalization_7[0][0]']  
                                                                                                  
 dropout_9 (Dropout)            (None, 240, 100)     0           ['dense_8[0][0]']                
                                                                                                  
 add_7 (Add)                    (None, 240, 100)     0           ['dropout_9[0][0]',              
                                                                  'dropout_9[0][0]']              
                                                                                                  
 layer_normalization_8 (LayerNo  (None, 240, 100)    200         ['add_7[0][0]']                  
 rmalization)                                                                                     
          

In [12]:
accuracy_callback = AccuracyCallback('accuracy')
f1_callback = AccuracyCallback('f1_m')


n_batch = 128
n_epochs = 500
ls =  'binary_crossentropy'
mtr = ['mse', 'accuracy',  f1_m]
opt = 'rmsprop'
model.compile(optimizer=opt, loss=ls, metrics=mtr)

In [13]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', 
        min_delta=0, patience=70, mode='max', restore_best_weights=True)

In [14]:
history = model.fit(x,y, validation_split=0.2, batch_size=n_batch, epochs=n_epochs,callbacks=[callback])

Epoch 1/500
129/129 [==============================] - 54s 322ms/step - loss: 6.6552 - output_1_loss: 0.6638 - output_2_loss: 0.6699 - output_3_loss: 0.6681 - output_4_loss: 0.6604 - output_5_loss: 0.6721 - output_6_loss: 0.6564 - output_7_loss: 0.6625 - output_8_loss: 0.6662 - output_9_loss: 0.6686 - output_10_loss: 0.6671 - output_1_mse: 0.2340 - output_1_accuracy: 0.9596 - output_1_f1_m: 0.0056 - output_2_mse: 0.2354 - output_2_accuracy: 0.9559 - output_2_f1_m: 0.0024 - output_3_mse: 0.2348 - output_3_accuracy: 0.9593 - output_3_f1_m: 0.0033 - output_4_mse: 0.2330 - output_4_accuracy: 0.9624 - output_4_f1_m: 0.0016 - output_5_mse: 0.2364 - output_5_accuracy: 0.9686 - output_5_f1_m: 3.7359e-04 - output_6_mse: 0.2317 - output_6_accuracy: 0.9622 - output_6_f1_m: 0.0052 - output_7_mse: 0.2337 - output_7_accuracy: 0.9596 - output_7_f1_m: 0.0046 - output_8_mse: 0.2346 - output_8_accuracy: 0.9709 - output_8_f1_m: 0.0012 - output_9_mse: 0.2348 - output_9_accuracy: 0.9668 - output_9_f1_m: 0.

Epoch 3/500
129/129 [==============================] - 35s 273ms/step - loss: 5.5036 - output_1_loss: 0.5521 - output_2_loss: 0.5525 - output_3_loss: 0.5517 - output_4_loss: 0.5508 - output_5_loss: 0.5494 - output_6_loss: 0.5509 - output_7_loss: 0.5520 - output_8_loss: 0.5482 - output_9_loss: 0.5494 - output_10_loss: 0.5466 - output_1_mse: 0.1801 - output_1_accuracy: 0.9696 - output_1_f1_m: 0.0000e+00 - output_2_mse: 0.1803 - output_2_accuracy: 0.9692 - output_2_f1_m: 0.0000e+00 - output_3_mse: 0.1799 - output_3_accuracy: 0.9715 - output_3_f1_m: 0.0000e+00 - output_4_mse: 0.1795 - output_4_accuracy: 0.9726 - output_4_f1_m: 0.0000e+00 - output_5_mse: 0.1787 - output_5_accuracy: 0.9792 - output_5_f1_m: 0.0000e+00 - output_6_mse: 0.1795 - output_6_accuracy: 0.9711 - output_6_f1_m: 0.0000e+00 - output_7_mse: 0.1800 - output_7_accuracy: 0.9693 - output_7_f1_m: 0.0000e+00 - output_8_mse: 0.1782 - output_8_accuracy: 0.9815 - output_8_f1_m: 0.0000e+00 - output_9_mse: 0.1788 - output_9_accuracy

Epoch 5/500
129/129 [==============================] - 35s 273ms/step - loss: 4.5738 - output_1_loss: 0.4605 - output_2_loss: 0.4611 - output_3_loss: 0.4595 - output_4_loss: 0.4586 - output_5_loss: 0.4552 - output_6_loss: 0.4590 - output_7_loss: 0.4606 - output_8_loss: 0.4536 - output_9_loss: 0.4555 - output_10_loss: 0.4503 - output_1_mse: 0.1366 - output_1_accuracy: 0.9696 - output_1_f1_m: 0.0000e+00 - output_2_mse: 0.1369 - output_2_accuracy: 0.9692 - output_2_f1_m: 0.0000e+00 - output_3_mse: 0.1362 - output_3_accuracy: 0.9715 - output_3_f1_m: 0.0000e+00 - output_4_mse: 0.1357 - output_4_accuracy: 0.9726 - output_4_f1_m: 0.0000e+00 - output_5_mse: 0.1341 - output_5_accuracy: 0.9792 - output_5_f1_m: 0.0000e+00 - output_6_mse: 0.1359 - output_6_accuracy: 0.9711 - output_6_f1_m: 0.0000e+00 - output_7_mse: 0.1367 - output_7_accuracy: 0.9693 - output_7_f1_m: 0.0000e+00 - output_8_mse: 0.1333 - output_8_accuracy: 0.9815 - output_8_f1_m: 0.0000e+00 - output_9_mse: 0.1342 - output_9_accuracy

Epoch 7/500
129/129 [==============================] - 35s 273ms/step - loss: 3.7952 - output_1_loss: 0.3840 - output_2_loss: 0.3847 - output_3_loss: 0.3826 - output_4_loss: 0.3813 - output_5_loss: 0.3762 - output_6_loss: 0.3823 - output_7_loss: 0.3841 - output_8_loss: 0.3740 - output_9_loss: 0.3768 - output_10_loss: 0.3692 - output_1_mse: 0.1029 - output_1_accuracy: 0.9696 - output_1_f1_m: 0.0000e+00 - output_2_mse: 0.1032 - output_2_accuracy: 0.9692 - output_2_f1_m: 0.0000e+00 - output_3_mse: 0.1022 - output_3_accuracy: 0.9715 - output_3_f1_m: 0.0000e+00 - output_4_mse: 0.1016 - output_4_accuracy: 0.9726 - output_4_f1_m: 0.0000e+00 - output_5_mse: 0.0992 - output_5_accuracy: 0.9792 - output_5_f1_m: 0.0000e+00 - output_6_mse: 0.1020 - output_6_accuracy: 0.9711 - output_6_f1_m: 0.0000e+00 - output_7_mse: 0.1029 - output_7_accuracy: 0.9693 - output_7_f1_m: 0.0000e+00 - output_8_mse: 0.0981 - output_8_accuracy: 0.9815 - output_8_f1_m: 0.0000e+00 - output_9_mse: 0.0994 - output_9_accuracy

Epoch 9/500
129/129 [==============================] - 36s 279ms/step - loss: 3.1553 - output_1_loss: 0.3215 - output_2_loss: 0.3222 - output_3_loss: 0.3196 - output_4_loss: 0.3180 - output_5_loss: 0.3111 - output_6_loss: 0.3195 - output_7_loss: 0.3217 - output_8_loss: 0.3083 - output_9_loss: 0.3119 - output_10_loss: 0.3018 - output_1_mse: 0.0779 - output_1_accuracy: 0.9696 - output_1_f1_m: 0.0000e+00 - output_2_mse: 0.0782 - output_2_accuracy: 0.9692 - output_2_f1_m: 0.0000e+00 - output_3_mse: 0.0770 - output_3_accuracy: 0.9715 - output_3_f1_m: 0.0000e+00 - output_4_mse: 0.0763 - output_4_accuracy: 0.9726 - output_4_f1_m: 0.0000e+00 - output_5_mse: 0.0731 - output_5_accuracy: 0.9792 - output_5_f1_m: 0.0000e+00 - output_6_mse: 0.0770 - output_6_accuracy: 0.9711 - output_6_f1_m: 0.0000e+00 - output_7_mse: 0.0780 - output_7_accuracy: 0.9693 - output_7_f1_m: 0.0000e+00 - output_8_mse: 0.0719 - output_8_accuracy: 0.9815 - output_8_f1_m: 0.0000e+00 - output_9_mse: 0.0735 - output_9_accuracy

...

Epoch 55/500
129/129 [==============================] - 35s 272ms/step - loss: 1.1611 - output_1_loss: 0.1362 - output_2_loss: 0.1377 - output_3_loss: 0.1294 - output_4_loss: 0.1255 - output_5_loss: 0.1010 - output_6_loss: 0.1309 - output_7_loss: 0.1371 - output_8_loss: 0.0920 - output_9_loss: 0.1040 - output_10_loss: 0.0672 - output_1_mse: 0.0295 - output_1_accuracy: 0.9696 - output_1_f1_m: 0.0000e+00 - output_2_mse: 0.0299 - output_2_accuracy: 0.9692 - output_2_f1_m: 0.0000e+00 - output_3_mse: 0.0277 - output_3_accuracy: 0.9715 - output_3_f1_m: 0.0000e+00 - output_4_mse: 0.0266 - output_4_accuracy: 0.9726 - output_4_f1_m: 0.0000e+00 - output_5_mse: 0.0203 - output_5_accuracy: 0.9792 - output_5_f1_m: 0.0000e+00 - output_6_mse: 0.0281 - output_6_accuracy: 0.9711 - output_6_f1_m: 0.0000e+00 - output_7_mse: 0.0297 - output_7_accuracy: 0.9693 - output_7_f1_m: 0.0000e+00 - output_8_mse: 0.0181 - output_8_accuracy: 0.9815 - output_8_f1_m: 0.0000e+00 - output_9_mse: 0.0211 - output_9_accurac

Epoch 57/500
129/129 [==============================] - 36s 280ms/step - loss: 1.1611 - output_1_loss: 0.1362 - output_2_loss: 0.1377 - output_3_loss: 0.1294 - output_4_loss: 0.1255 - output_5_loss: 0.1010 - output_6_loss: 0.1309 - output_7_loss: 0.1371 - output_8_loss: 0.0920 - output_9_loss: 0.1040 - output_10_loss: 0.0672 - output_1_mse: 0.0295 - output_1_accuracy: 0.9696 - output_1_f1_m: 0.0000e+00 - output_2_mse: 0.0299 - output_2_accuracy: 0.9692 - output_2_f1_m: 0.0000e+00 - output_3_mse: 0.0277 - output_3_accuracy: 0.9715 - output_3_f1_m: 0.0000e+00 - output_4_mse: 0.0266 - output_4_accuracy: 0.9726 - output_4_f1_m: 0.0000e+00 - output_5_mse: 0.0203 - output_5_accuracy: 0.9792 - output_5_f1_m: 0.0000e+00 - output_6_mse: 0.0281 - output_6_accuracy: 0.9711 - output_6_f1_m: 0.0000e+00 - output_7_mse: 0.0297 - output_7_accuracy: 0.9693 - output_7_f1_m: 0.0000e+00 - output_8_mse: 0.0181 - output_8_accuracy: 0.9815 - output_8_f1_m: 0.0000e+00 - output_9_mse: 0.0211 - output_9_accurac

Epoch 59/500
129/129 [==============================] - 35s 275ms/step - loss: 1.1611 - output_1_loss: 0.1362 - output_2_loss: 0.1377 - output_3_loss: 0.1294 - output_4_loss: 0.1255 - output_5_loss: 0.1010 - output_6_loss: 0.1309 - output_7_loss: 0.1371 - output_8_loss: 0.0920 - output_9_loss: 0.1040 - output_10_loss: 0.0672 - output_1_mse: 0.0295 - output_1_accuracy: 0.9696 - output_1_f1_m: 0.0000e+00 - output_2_mse: 0.0299 - output_2_accuracy: 0.9692 - output_2_f1_m: 0.0000e+00 - output_3_mse: 0.0277 - output_3_accuracy: 0.9715 - output_3_f1_m: 0.0000e+00 - output_4_mse: 0.0266 - output_4_accuracy: 0.9726 - output_4_f1_m: 0.0000e+00 - output_5_mse: 0.0203 - output_5_accuracy: 0.9792 - output_5_f1_m: 0.0000e+00 - output_6_mse: 0.0281 - output_6_accuracy: 0.9711 - output_6_f1_m: 0.0000e+00 - output_7_mse: 0.0297 - output_7_accuracy: 0.9693 - output_7_f1_m: 0.0000e+00 - output_8_mse: 0.0181 - output_8_accuracy: 0.9815 - output_8_f1_m: 0.0000e+00 - output_9_mse: 0.0211 - output_9_accurac

Epoch 61/500
129/129 [==============================] - 36s 279ms/step - loss: 1.1611 - output_1_loss: 0.1362 - output_2_loss: 0.1377 - output_3_loss: 0.1294 - output_4_loss: 0.1255 - output_5_loss: 0.1010 - output_6_loss: 0.1309 - output_7_loss: 0.1371 - output_8_loss: 0.0920 - output_9_loss: 0.1040 - output_10_loss: 0.0672 - output_1_mse: 0.0295 - output_1_accuracy: 0.9696 - output_1_f1_m: 0.0000e+00 - output_2_mse: 0.0299 - output_2_accuracy: 0.9692 - output_2_f1_m: 0.0000e+00 - output_3_mse: 0.0277 - output_3_accuracy: 0.9715 - output_3_f1_m: 0.0000e+00 - output_4_mse: 0.0266 - output_4_accuracy: 0.9726 - output_4_f1_m: 0.0000e+00 - output_5_mse: 0.0203 - output_5_accuracy: 0.9792 - output_5_f1_m: 0.0000e+00 - output_6_mse: 0.0281 - output_6_accuracy: 0.9711 - output_6_f1_m: 0.0000e+00 - output_7_mse: 0.0297 - output_7_accuracy: 0.9693 - output_7_f1_m: 0.0000e+00 - output_8_mse: 0.0181 - output_8_accuracy: 0.9815 - output_8_f1_m: 0.0000e+00 - output_9_mse: 0.0211 - output_9_accurac

KeyboardInterrupt: 